<a href="https://colab.research.google.com/github/the-SQuAD-squad/IR/blob/electra/IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DA FARE: 
-Sistemare dataset in modo che le coppie domanda passaggio siano giuste
-Provare modello dense, senza trainare Bert

In [5]:
#@title Init { form-width: "25%" }
import os
import random
import math
import numpy as np
import tensorflow as tf
import json
import pandas as pd
import re
import string
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

!pip install tokenizers

from tokenizers import BertWordPieceTokenizer

!pip install transformers
import transformers
from transformers import AutoTokenizer
pd.set_option('display.max_colwidth', -1)

# fix random seeds
seed_value = 42 #@param {type:"integer"}

os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

tf.compat.v1.set_random_seed(seed_value)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# BERT params
max_seq_length = 512
# Huggingface bert and associated tokenizer
pretrained_model_str = "google/electra-small-discriminator"
bert_hf_layer = transformers.TFElectraModel.from_pretrained(
    pretrained_model_str, output_attentions=True)

#pretrained_model_str = "roberta-base"
#bert_hf_layer = transformers.TFRobertaModel.from_pretrained(
    #pretrained_model_str, output_attentions=True)

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_str)

!nvidia-smi

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraModel: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFElectraModel were initialized from the model checkpoint at google/electra-small-discriminator.
If your task is similar to the task th

Tue Feb 23 15:51:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W /  70W |    540MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
#@title df creation { form-width: "25%" }

# the official dataset is identical to the provided one
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json -O training_set.json

with open("training_set.json", "r") as f:
    json_file = json.load(f)
data = json_file["data"]

rows = []
for document in data:
  for par in document['paragraphs']:
    for qas in par['qas']:
      rows.append({
        'id' : qas['id'],
        'title': document["title"],
        'passage': par['context'],
        'question' : qas['question'],
        'answer_idx' : (qas['answers'][0]['answer_start'], 
                    qas['answers'][0]['answer_start'] + len(qas['answers'][0]['text'])),
        'answer_text' : qas['answers'][0]['text']
      })

df_original = pd.DataFrame(rows)

--2021-02-23 15:51:30--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘training_set.json’

training_set.json   100%[===================>]  28.88M  78.3MB/s    in 0.4s    

2021-02-23 15:51:30 (78.3 MB/s) - ‘training_set.json’ saved [30288272/30288272]



In [7]:
#@title clean dataset { form-width: "25%" }

!gcloud config set project feisty-mechanic-221914
!gsutil cp gs://squad_squad/error_IDs.txt ./error_IDs.txt

with open("error_IDs.txt", "r") as f:
    unwanted_id = f.read()

unwanted_id = unwanted_id.split("\n")[:-1]
df_bert = df_original.set_index('id')
df_bert = df_bert.drop(unwanted_id)
df_bert.head()

Updated property [core/project].
Copying gs://squad_squad/error_IDs.txt...
/ [1 files][  5.7 KiB/  5.7 KiB]                                                
Operation completed over 1 objects/5.7 KiB.                                      


,title,passage,question,answer_idx,answer_text
id,,,,,
5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,"(515, 541)",Saint Bernadette Soubirous
5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",What is in front of the Notre Dame Main Building?,"(188, 213)",a copper statue of Christ
5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",The Basilica of the Sacred heart at Notre Dame is beside to which structure?,"(279, 296)",the Main Building
5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",What is the Grotto at Notre Dame?,"(381, 420)",a Marian place of prayer and reflection
5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary rep

In [8]:
#@title BERT preprocessing { form-width: "25%" }
from tqdm import tqdm

def preprocess_bert(text):
    tokenized_text = tokenizer(list(text), padding="max_length", max_length=max_seq_length, truncation=True)
    return tokenized_text.input_ids

df_bert_preprocessed = df_bert.copy()
# pre-process passage and question text
print("Preprocessing passage...")
df_bert_preprocessed['passage'] = preprocess_bert(df_bert['passage'])
print("Preprocessing question...")
df_bert_preprocessed['question'] = preprocess_bert(df_bert['question'])
df_bert_preprocessed.head()

Preprocessing passage...
Preprocessing question...


,title,passage,question,answer_idx,answer_text
id,,,,,
5733be284776f41900661182,University_of_Notre_Dame,"[101, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, ...]","[101, 2000, 3183, 2106, 1996, 6261, 2984, 9382, 3711, 1999, 8517, 1999, 10223, 26371, 2605, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","(515, 541)",Saint Bernadette Soubirous
5733be284776f4190066117f,University_of_Notre_Dame,"[101, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, ...]","[101, 2054, 2003, 1999, 2392, 1997, 1996, 10289, 8214, 2364, 2311, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","(188, 213)",a copper statue of Christ
5733be284776f41900661180,University_of_Notre_Dame,"[101, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, ...]","[101, 1996, 13546, 1997, 1996, 6730, 2540, 2012, 10289, 8214, 2003, 3875, 2000, 2029, 3252, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","(279, 296)",the Main Building
5733be284776f41900661181,University_of_Notre_Dame,"[101, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, ...]","[101, 2054, 2003, 1996, 24665, 23052, 2012, 10289, 8214, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [56]:
# negative mining loss oriented data loader

class IRDataset(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size):
        self.df = df
        self.batch_size = batch_size
        self.groups = df.groupby("title").groups

    def __len__(self):
        return 2 * math.floor(len(self.df)/self.batch_size)                #####################################################################################################

    def __getitem__(self, idx):
        half = self.batch_size//2
        q = self.df['question'].iloc[idx * half : (idx + 1) * half].tolist()
        p = self.df['passage'].iloc[idx * half : (idx + 1) * half].tolist()
        attention_mask_q = np.stack(q)
        attention_mask_q[attention_mask_q != 0] = 1
        attention_mask_p = np.stack(p)
        attention_mask_p[attention_mask_p != 0] = 1
        # if len(p) == 0 or len(q) == 0:
        #     return ([], []), []

        # original = df_bert.loc[self.df.index]
        # unique_passage = df_bert.loc[self.df.index]['passage'].unique()
        # print(unique_passage)
        notp = []
        for index in range(idx * half,(idx + 1) * half):
            title = self.df["title"].iloc[index]
            passage = self.df["passage"].iloc[index]
            group = self.groups[title]
            new_group = []
            for id in group:
                if self.df["passage"][id] != passage:
                    new_group.append(id)
            notp.append(self.df["passage"][np.random.choice(new_group)])
        #print(f"notp: {len(notp)}")
        #not_p = groups[]
        # DA SISTEMARE; CI POTREBBERO ESSERE ANCORA I PASSAGGI GIUSTI PERCHè SONO RIPETUTI
        all_not_p = list(range(0,idx * half)) + list(range((idx + 1) * half, len(self.df)))
        #print(all_not_p)
        rnd = np.random.choice(all_not_p, half)                                #####################################################################################################
        #print(f"rnd is {rnd}")
        #notp = self.df['passage'].iloc[rnd].tolist()
        attention_mask_not_p = np.stack(notp)
        attention_mask_not_p[attention_mask_not_p != 0] = 1
        #print(f"notp is {notp}")
        y = half*[1]+half*[0]
        #print(np.stack(p+notp))
        return (np.stack(p+notp),np.vstack((attention_mask_p,attention_mask_not_p)), np.stack(q+q),np.vstack((attention_mask_q,attention_mask_q))), np.array(y)
        #y = half*[1]+half*[1]
        #return (np.stack(p+p),np.vstack((attention_mask_p,attention_mask_p)), np.stack(q+q),np.vstack((attention_mask_q,attention_mask_q))), np.array(y)

split_value = 0.1 
val_dim = int(len(df_bert_preprocessed['title'].unique()) * split_value)
val_titles = np.random.choice(df_bert_preprocessed['title'].unique(), size=val_dim, replace=False)

df_bert_val = df_bert_preprocessed[df_bert_preprocessed['title'].isin(val_titles)]
df_bert_train = df_bert_preprocessed[~(df_bert_preprocessed['title'].isin(val_titles))]

In [50]:
seq_train = IRDataset(df_bert_train, batch_size)
seq_train.__getitem__(0)

notp: 8


((array([[  101,  6549,  2135, ...,     0,     0,     0],
         [  101,  6549,  2135, ...,     0,     0,     0],
         [  101,  6549,  2135, ...,     0,     0,     0],
         ...,
         [  101,  5181,  2036, ...,     0,     0,     0],
         [  101,  4199,  2018, ...,     0,     0,     0],
         [  101, 10556, 17112, ...,     0,     0,     0]]),
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]]),
  array([[  101,  2000,  3183, ...,     0,     0,     0],
         [  101,  2054,  2003, ...,     0,     0,     0],
         [  101,  1996, 13546, ...,     0,     0,     0],
         ...,
         [  101,  2043,  2106, ...,     0,     0,     0],
         [  101,  2129,  2411, ...,     0,     0,     0],
         [  101,  2054,  2003, ...,     0,     0,     0]]),
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1,

{'2008_Sichuan_earthquake': ['56cdca7862d2951400fa6826', '56cdca7862d2951400fa6827', '56cdca7862d2951400fa6828', '56d4f9902ccc5a1400d833c0', '56d4f9902ccc5a1400d833c1', '56d4f9902ccc5a1400d833c2', '56d4f9902ccc5a1400d833c3', '56d4f9902ccc5a1400d833c4', '56cdcb2c62d2951400fa6830', '56cdcb2c62d2951400fa6833', '56cdcb2c62d2951400fa6834', '56d4fb6f2ccc5a1400d833d4', '56d4fb6f2ccc5a1400d833d5', '56d4fb6f2ccc5a1400d833d6', '56d4fb6f2ccc5a1400d833d7', '56cdcbb762d2951400fa683a', '56cdcbb762d2951400fa683b', '56cdcbb762d2951400fa683c', '56cdcbb762d2951400fa683d', '56cdcbb762d2951400fa683e', '56d4fca52ccc5a1400d833de', '56d4fca52ccc5a1400d833df', '56d4fca52ccc5a1400d833e0', '56d4fca52ccc5a1400d833e1', '56d4fca52ccc5a1400d833e2', '56cdcc5562d2951400fa6845', '56cdcc5562d2951400fa6847', '56cdcc5562d2951400fa6848', '56d4fe332ccc5a1400d833e8', '56d4fe332ccc5a1400d833eb', '56d4fe332ccc5a1400d833ec', '56cdccd962d2951400fa684e', '56cdccd962d2951400fa684f', '56cdccd962d2951400fa6850', '56cdccd962d2951400

In [ ]:
df_bert_train.head()
df_bert_train["passage"].loc["5733be284776f41900661182"]

In [60]:
#@title model definition { form-width: "25%" }
import tensorflow as tf
from tensorflow.keras.losses import cosine_similarity, MSE, mae

input_passage_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_passage_ids')
input_passage_attention = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_passage_attention')
input_question_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_question_ids')
input_question_attention = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_question_attention')

#bert_hf_layer = transformers.TFElectraModel.from_pretrained(pretrained_model_str, output_attentions=True)
bert_hf_layer = transformers.TFElectraModel.from_pretrained(pretrained_model_str)

passage_encoding = bert_hf_layer(input_ids=[input_passage_ids, input_passage_attention]).last_hidden_state[:,0]
question_encoding = bert_hf_layer(input_ids=[input_question_ids, input_question_attention]).last_hidden_state[:,0]

output = tf.stack([passage_encoding, question_encoding],axis=-1)


######################
concat_quest_pass = tf.keras.layers.concatenate([question_encoding, passage_encoding])

output = layers.Dense(1, activation= "sigmoid")(concat_quest_pass)
######################

model = keras.Model(inputs=[input_passage_ids, input_passage_attention, input_question_ids, input_question_attention], 
                    outputs=output,
                    name="BERT_IR")

def similarity_loss(y_true, y_pred):
    cs = (-cosine_similarity(y_pred[:,:,0], y_pred[:,:,1]) +1) /2  # cs is 1 when the two are similar, 0 otherwise
    cs = tf.expand_dims(cs, axis=1)
    # print(y_true)
    # print()
    # print(y_pred)
    # print()
    # print(cs)
    return MSE(y_true, cs)
    #return cosine_similarity(y_pred[:,:,0], y_pred[:,:,1])
    #return mae(y_true,cs)

#optimizer = keras.optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
optimizer = keras.optimizers.Adam()

model.summary(line_length=150)

Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraModel: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFElectraModel were initialized from the model checkpoint at google/electra-small-discriminator.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further training.


Model: "BERT_IR"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_question_ids (InputLayer)                  [(None, 512)]                    0                                                                   
______________________________________________________________________________________________________________________________________________________
input_question_attention (InputLayer)            [(None, 512)]                    0                                                                   
______________________________________________________________________________________________________________________________________________________
input_passage_ids (InputLayer)                   [(None, 512)]               

In [11]:
### TO SAVE THE WEIGHTS TO DRIVE ###
from google.colab import drive
import os.path
drive.mount('/content/drive')  # mount the personal drive folder

# create a target folder if it doesn't exist
path = "drive/My Drive/weights_IR"
if not os.path.exists(path):
    os.mkdir(path)

Mounted at /content/drive


In [ ]:
### LOAD WEIGHTS FROM DRIVE ###
model.load_weights("/content/drive/MyDrive/weights_IR")

In [ ]:
#@title train { form-width: "25%" }

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
batch_size = 16
seq_train = IRDataset(df_bert_train, batch_size)
seq_val = IRDataset(df_bert_val, batch_size)
epochs = 200
steps_per_epoch = 20
saveDir = os.path.join(os.getcwd(), 'saved_models')
if not os.path.isdir(saveDir):
    os.makedirs(saveDir)
#chkpt = saveDir + '/squad_check.hdf5'
chkpt = path + '/weights_IR.hdf5'

ENABLE_WANDB = False        #@param {type:"boolean"}
wandb_experiment_name = "HF_" + bert_hf_layer.name  #@param {type: "string"}
if ENABLE_WANDB:
    !pip install wandb > /dev/null
    !wandb login
    import wandb
    from wandb.keras import WandbCallback
    wandb.init(project="SQUAD", name=wandb_experiment_name)
    wandb.config.batch_size = batch_size
    wandb.config.epochs = epochs
    
es_cb = EarlyStopping(monitor='val_loss', patience=2,verbose=1, mode='auto')
cp_cb = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, 
                        save_best_only=False, mode='auto', 
                        save_weights_only=True)

callbacks = [es_cb, cp_cb]

if ENABLE_WANDB:
    callbacks.append(WandbCallback(log_batch_frequency=10,
                                   save_weights_only=True))

tf.keras.backend.clear_session()

#model.compile(optimizer=optimizer, loss=similarity_loss)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics = "accuracy")
history = model.fit(seq_train, epochs=epochs,callbacks=callbacks, validation_data=seq_val,batch_size=batch_size)

Epoch 1/200
 313/9886 [..............................] - ETA: 2:10:30 - loss: 0.7535 - accuracy: 0.4992

In [14]:
# CELLA DI PROVA INFERENCE
#@title inference model PROVA { form-width: "25%" }
from sklearn.metrics.pairwise import cosine_similarity

batch_size = 16

all_encodings = []

for idx in tqdm(range(len(df_bert_val)//batch_size)):
    questions = df_bert_val['question'].iloc[idx * batch_size : (idx + 1) * batch_size].tolist()
    attention_mask_questions = np.stack(questions)
    attention_mask_questions[attention_mask_questions != 0] = 1
    passages = df_bert_val['passage'].iloc[idx * batch_size : (idx + 1) * batch_size].tolist()
    attention_mask_passages = np.stack(passages)
    attention_mask_passages[attention_mask_passages != 0] = 1
    encodings = model.predict([np.stack(passages), attention_mask_passages, np.stack(questions), attention_mask_questions])
    all_encodings.append(encodings)
    break
    # encoded_val_passage[i*batch_size : (i+1)*batch_size] = inf_model(passages[i*batch_size : (i+1)*batch_size])
    # encoded_val_question[i*batch_size : (i+1)*batch_size] = inf_model(questions[i*batch_size : (i+1)*batch_size])

#print(encodings.shape)

# all_encodings_arr = np.vstack(all_encodings)
# encoding_passages = all_encodings_arr[:,:,0]
# encoding_questions = all_encodings_arr[:,:,1]

encoding_passages = encodings[:,:,0]
encoding_questions = encodings[:,:,1]
first_passage_encoding = encoding_passages[0]
first_question_encoding = encoding_questions[0]
#print(first_passage_encoding.reshape(1,-1).shape)
print(cosine_similarity(first_question_encoding.reshape(1,-1),first_passage_encoding.reshape(1,-1)))
#print(cosine_similarity(encoding_questions, encoding_passages))

  0%|          | 0/446 [00:00<?, ?it/s]

[[-0.0657178]]


In [ ]:
# SECONDA CELLA PROVA INFERENCE
cos_sim = cosine_similarity(encoding_questions, encoding_passages)
print(np.argmax(cos_sim[878]))
ok=0
for i in range(7136):   # i is the question index
    best = np.argmax(cos_sim[i])
    if np.array_equal(df_bert_val.iloc[i].passage, df_bert_val.iloc[best].passage):
        ok+=1

1673


In [ ]:
len(df_bert_val)

7148

In [ ]:
print(ok/7136)

0.0008408071748878924


In [ ]:
print(cos_sim[0][:100])

[0.94073474 0.94073474 0.94073474 0.94073474 0.94073474 0.94073474
 0.94073474 0.94073474 0.940735   0.940735   0.940735   0.940735
 0.940735   0.940735   0.940735   0.940735   0.940735   0.940735
 0.940735   0.940735   0.9222013  0.9222013  0.9222013  0.9222013
 0.9222013  0.9222013  0.9222013  0.9222013  0.9222013  0.9222013
 0.9222013  0.9222013  0.8756785  0.8756785  0.8756785  0.8756785
 0.8756785  0.8756785  0.8756785  0.8756785  0.8756785  0.8756785
 0.8756785  0.8756785  0.7282998  0.7282998  0.7282998  0.7282998
 0.7282998  0.7282998  0.7282998  0.7282998  0.7282998  0.7282998
 0.7282998  0.7282998  0.7282998  0.71472895 0.71472895 0.71472895
 0.71472895 0.71472895 0.71472895 0.71472895 0.71472895 0.71472895
 0.71472895 0.71472895 0.95767933 0.95767933 0.95767933 0.95767933
 0.95767933 0.95767933 0.95767933 0.95767933 0.95767933 0.95767933
 0.95767933 0.95767933 0.95767933 0.96482575 0.96482575 0.96482575
 0.96482575 0.96482575 0.96482575 0.96482575 0.96482575 0.96482575
 0.96

In [ ]:
df_bert_val.iloc[:100]

,title,passage,question,answer_idx,answer_text
id,,,,,
56be85543aeaaa14008c9063,Beyoncé,"[101, 20773, 21025, 19358, 22815, 1011, 5708, 1006, 1013, 12170, 23432, 29715, 3501, 29678, 12325, 29685, 1013, 10506, 1011, 10930, 2078, 1011, 2360, 1007, 1006, 2141, 2244, 1018, 1010, 3261, 1007, 2003, 2019, 2137, 3220, 1010, 6009, 1010, 2501, 3135, 1998, 3883, 1012, 2141, 1998, 2992, 1999, 5395, 1010, 3146, 1010, 2016, 2864, 1999, 2536, 4823, 1998, 5613, 6479, 2004, 1037, 2775, 1010, 1998, 3123, 2000, 4476, 1999, 1996, 2397, 4134, 2004, 2599, 3220, 1997, 1054, 1004, 1038, 2611, 1011, 2177, 10461, 1005, 1055, 2775, 1012, 3266, 2011, 2014, 2269, 1010, 25436, 22815, 1010, 1996, 2177, 2150, 2028, 1997, 1996, ...]","[101, 2043, 2106, 20773, 2707, 3352, 2759, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","(269, 286)",in the late 1990s
56be85543aeaaa14008c9065,Beyoncé,"[101, 20773, 21025, 19358, 22815, 1011, 5708, 1006, 1013, 12170, 23432, 29715, 3501, 29678, 12325, 29685, 1013, 10506, 1011, 10930, 2078, 1011, 2360, 1007, 1006, 2141, 2244, 1018, 1010, 3261, 1007, 2003, 2019, 2137, 3220, 1010, 6009, 1010, 2501, 3135, 1998, 3883, 1012, 2141, 1998, 2992, 1999, 5395, 1010, 3146, 1010, 2016, 2864, 1999, 2536, 4823, 1998, 5613, 6479, 2004, 1037, 2775, 1010, 1998, 3123, 2000, 4476, 1999, 1996, 2397, 4134, 2004, 2599, 3220, 1997, 1054, 1004, 1038, 2611, 1011, 2177, 10461, 1005, 1055, 2775, 1012, 3266, 2011, 2014, 2269, 1010, 25436, 22815, 1010, 1996, 2177, 2150, 2028, 1997, 1996, ...]","[101, 2054, 2752, 2106, 20773, 5566, 1999, 2043, 2016, 2001, 3652, 2039, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","(207, 226)",singing and dancing
56be85543aeaaa14008c9066,Beyoncé,"[101, 20773, 21025, 19358, 22815, 1011, 5708, 1006, 1013, 12170, 23432, 29715, 3501, 29678, 12325, 29685, 1013, 10506, 1011, 10930, 2078, 1011, 2360, 1007, 1006, 2141, 2244, 1018, 1010, 3261, 1007, 2003, 2019, 2137, 3220, 1010, 6009, 1010, 2501, 3135, 1998, 3883, 1012, 2141, 1998, 2992, 1999, 5395, 1010, 3146, 1010, 2016, 2864, 1999, 2536, 4823, 1998, 5613, 6479, 2004, 1037, 2775, 1010, 1998, 3123, 2000, 4476, 1999, 1996, 2397, 4134, 2004, 2599, 3220, 1997, 1054, 1004, 1038, 2611, 1011, 2177, 10461, 1005, 1055, 2775, 1012, 3266, 2011, 2014, 2269, 1010, 25436, 22815, 1010, 1996, 2177, 2150, 2028, 1997, 1996, ...]","[101, 2043, 2106, 20773, 2681, 10461, 1005, 1055, 2775, 1998, 2468, 1037, 3948, 3220, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","(526, 530)",2003
56bf6b0f3aeaaa14008c9601,Beyoncé,"[101, 20773, 21025, 19358, 22815, 1011, 5708, 1006, 1013, 12170, 23432, 29715, 3501, 29678, 12325, 29685, 1013, 10506, 1011, 10930, 2078, 1011, 2360, 1007, 1006, 2141, 2244, 1018, 1010, 3261, 1007, 2003, 2019, 2137, 3220, 1010, 6009, 1010, 2501, 3135, 1998, 3883, 1012, 2141, 1998, 2992, 1999, 5395, 1010, 3146, 1010, 2016, 2864, 1999, 2536, 4823, 1998, 5613, 6479, 2004, 1037, 2775, 1010, 1998, 3123, 2000, 4476, 1999, 1996, 2397, 4134, 2004, 2599, 3220, 1997, 1054, 1004, 1038, 2611, 1011, 2177, 10461, 1005, 1055, 2775, 1012, 3266, 2011, 2014, 2269, 1010, 25436, 22815, 1010, 1996, 2177, 2150, 2028, 1997, 1996, ...]","[101, 1999, 2054, 2103, 1998, 2110, 2106, 20773, 4982, 2039, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# TERZA CELLA PROVA INFERENCE
df_bert_val.iloc[3034]
df_bert.loc["571a48c110f8ca1400304fc7"]
# When did Beyonce start becoming popular? 
# Who began producing Oswald cartoons for Universal in 1929?  
#What is another term for specific formal prescriptions?   

title          Memory                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
passage        The working memory model explains many practical observations, such as why it is easier to do two different tasks (one verbal and one visual) than two similar tasks (e.g., two visual), and the aforementioned word-length effect. However, the concept of a central executive as noted here has been criticised as inadequate and vague.[citation needed] Working memory is also the premise for what allows us to do eve

In [ ]:
#@title inference model { form-width: "25%" }

input_txt_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32)
output = bert_hf_layer(input_ids=input_txt_ids).last_hidden_state[:,0]
inf_model = keras.Model(inputs=input_txt_ids, outputs=output, name="BERT_IR")

encoded_val_passage = np.zeros((len(df_bert_val), 256))
encoded_val_question = np.zeros((len(df_bert_val), 256))
passages = np.stack(df_bert_val['passage'])
questions = np.stack(df_bert_val['question'])


for i in tqdm(range(len(df_bert_val)//batch_size)):
    encoded_val_passage[i*batch_size : (i+1)*batch_size] = inf_model(passages[i*batch_size : (i+1)*batch_size])
    encoded_val_question[i*batch_size : (i+1)*batch_size] = inf_model(questions[i*batch_size : (i+1)*batch_size])

 61%|██████    | 325/531 [01:29<00:57,  3.60it/s]

KeyboardInterrupt: ignored

In [ ]:
#@title kNN cosine similarity { form-width: "25%" }

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

k=1
tree = NearestNeighbors(n_neighbors=k, metric='cosine')
tree.fit(encoded_val_passage)

results = tree.kneighbors(encoded_val_question, n_neighbors=k, return_distance=False)

ok=0
for i in range(len(df_bert_val)):
    for j in range(k):
        if np.array_equal(passages[results[i,j]], passages[i]):
            ok+=1

ok/len(df_bert_val)*100

0.05595970900951316

In [ ]:
results[0]

array([1213, 1210, 1212, 1211, 1675])

In [ ]:
!pip3 install wikipedia-api

import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')
cat = wiki_wiki.page("Category:Physics")
print("Category members: Category:Physics")
for p in cat.categorymembers.values():
  if p.namespace == wikipediaapi.Namespace.CATEGORY:
    # it is category, so you have to make decision
    # if you want to fetch also text from pages that belong
    # to this category
    print(p)
  elif p.namespace == wikipediaapi.Namespace.MAIN:
    # it is page => we can get text
    print(p)
    print(p.text)

def print_categorymembers(categorymembers, level=0, max_level=3):
        for c in categorymembers.values():
            print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
            if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
                print_categorymembers(c.categorymembers, level=level + 1, max_level=max_level)

cat = wiki_wiki.page("Category:Artificial intelligence")
print_categorymembers(cat.categorymembers)

wiki_wiki = wikipediaapi.Wikipedia('en')

page_py = wiki_wiki.page('Python_(programming_language)')
print(page_py.text)